In [1]:
# how different are relapse to emc92 genes? plot them in the relapse figure

# Predict states based on gene subset ranking

In [37]:
import json,mygene,pandas,numpy
import scipy,scipy.stats

import matplotlib,matplotlib.pyplot
matplotlib.rcParams.update({'font.size':18,'font.family':'Arial','xtick.labelsize':14,'ytick.labelsize':14,'axes.labelsize':18})
matplotlib.rcParams['pdf.fonttype']=42

In [3]:
def histogrammer(x):

    # get number of bins based on Rice's rule
    rice=int((len(x)**(1/3))*2)
    print('\t number of bins according to Rice: {}'.format(rice))

    counts,edges=numpy.histogram(x,bins=rice)
    half=(edges[1]-edges[0])/2
    centers=edges[:-1]+half

    return centers,counts

# 0. user defined variables

In [4]:
tpms_file='/Volumes/GoogleDrive/My Drive/projects/MINER/shared/MINER/data/expression/MMRF_CoMMpass_IA9_E74GTF_Salmon_Gene_TPM.txt'
zscore_file='/Volumes/GoogleDrive/My Drive/projects/MINER/shared/MINER/data/expression/IA12Zscore.csv'
emc92_annotation_file='/Volumes/GoogleDrive/My Drive/projects/MINER/data/emc92.csv'
transcriptional_states_file='/Users/alomana/Google Drive File Stream/My Drive/projects/MINER/shared/MINER/results_minCorrelation_0o2_50_allFiles/transcriptional_states.json'

# 1. read data

## 1.1 read gene expression data

In [5]:
tpms=pandas.read_csv(tpms_file,header=0,index_col=0,sep="\t")
print(tpms.shape)
tpms.head()

(57997, 734)


,MMRF_2490_1_BM,MMRF_2499_1_BM,MMRF_1089_1_BM,MMRF_1778_1_BM,MMRF_1777_1_BM,MMRF_1153_1_BM,MMRF_1392_1_BM,MMRF_1325_1_BM,MMRF_1846_1_BM,MMRF_1935_1_BM,...,MMRF_1847_1_BM,MMRF_1787_1_BM,MMRF_1048_1_BM,MMRF_2437_1_BM,MMRF_1242_5_BM,MMRF_1497_1_BM,MMRF_1380_2_BM,MMRF_1700_1_BM,MMRF_1918_1_BM,MMRF_1670_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ERCC-00170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.2. read EMC92 genes

In [6]:
df=pandas.read_csv(emc92_annotation_file,header=None,sep=',')
print(df.shape)
df.head()

(92, 2)


,0,1
0,202728_s_at,LTBP1
1,239054_at,SFMBT1
2,208942_s_at,SEC62
3,208747_s_at,C1S
4,202542_s_at,AIMP1


In [7]:
genes=df[1].to_list()
print(genes,len(genes))

['LTBP1', 'SFMBT1', 'SEC62', 'C1S', 'AIMP1', 'ZBTB25', 'ACVR2A', 'S100A6', 'ITGA6', 'HMGB3', 'BCS1L', 'LOC100271836', 'PPP2R1B', 'PCDHB7', 'DYNLRB2', 'IL7R', 'DNAJB9', 'RAB2A', 'FGFR3', 'TSPAN16', 'DUX4', 'FAM49A', 'ROBO3', 'SUN1', 'TARBP1', 'TMEM97', 'EST/BX647543', 'STAT1', 'SLC17A5', 'DHRS9', 'MAGEA6', '---', 'ATPBD4', 'GRB14', 'GABRA4', 'NOP56', 'ESPL1', 'C15orf38', 'CCRL1', 'MARCKS', 'EST/BE568408', 'EHBP1L1', 'ST13', 'C18orf10', 'TOP2A', 'PFKL', 'ATP6V0E1', 'SAR1B', 'CDH22', 'RPS28', 'MRPL41', 'RPS4X', 'SLC30A7', 'ARPC5', 'FANCF', 'TUBB', 'TRAM1', 'ITM2B', 'MCM2', 'TUBB', 'NPC2', 'HMGN5', 'DTL', 'ANGEL2', 'ZNF252', 'NCAPG', 'BIRC5', 'SEPT11', 'FTL', 'HNRNPK', 'SPATS2L', 'PGM2', 'GGPS1', 'DONSON', 'KIF4A', 'AK2', 'FANCI', 'POLQ', 'C11orf85', 'MCM6', 'CENPE', 'NUF2', 'LBR', 'RPS11', 'SYF2', 'MCM3', 'ZWINT', 'NCUBE1', 'DARS2', 'ARL8B', 'DHFR', 'SPAG5'] 92


In [8]:
genes[genes.index('EST/BX647543')]='KCNJ5'
genes[genes.index('EST/BE568408')]='BBOX1-AS1'
genes[genes.index('LOC100271836')]='SMG1P3'
genes[genes.index('ZNF252')]='ZNF252P'
genes.remove('---')
print(genes,len(genes))
print(len(list(set(genes))))
set([x for x in genes if genes.count(x) > 1])

['LTBP1', 'SFMBT1', 'SEC62', 'C1S', 'AIMP1', 'ZBTB25', 'ACVR2A', 'S100A6', 'ITGA6', 'HMGB3', 'BCS1L', 'SMG1P3', 'PPP2R1B', 'PCDHB7', 'DYNLRB2', 'IL7R', 'DNAJB9', 'RAB2A', 'FGFR3', 'TSPAN16', 'DUX4', 'FAM49A', 'ROBO3', 'SUN1', 'TARBP1', 'TMEM97', 'KCNJ5', 'STAT1', 'SLC17A5', 'DHRS9', 'MAGEA6', 'ATPBD4', 'GRB14', 'GABRA4', 'NOP56', 'ESPL1', 'C15orf38', 'CCRL1', 'MARCKS', 'BBOX1-AS1', 'EHBP1L1', 'ST13', 'C18orf10', 'TOP2A', 'PFKL', 'ATP6V0E1', 'SAR1B', 'CDH22', 'RPS28', 'MRPL41', 'RPS4X', 'SLC30A7', 'ARPC5', 'FANCF', 'TUBB', 'TRAM1', 'ITM2B', 'MCM2', 'TUBB', 'NPC2', 'HMGN5', 'DTL', 'ANGEL2', 'ZNF252P', 'NCAPG', 'BIRC5', 'SEPT11', 'FTL', 'HNRNPK', 'SPATS2L', 'PGM2', 'GGPS1', 'DONSON', 'KIF4A', 'AK2', 'FANCI', 'POLQ', 'C11orf85', 'MCM6', 'CENPE', 'NUF2', 'LBR', 'RPS11', 'SYF2', 'MCM3', 'ZWINT', 'NCUBE1', 'DARS2', 'ARL8B', 'DHFR', 'SPAG5'] 91
90


{'TUBB'}

In [9]:
# convert gene IDs into ensembl IDs
emc92_ensembl_conversion={}
for gene in genes:
    emc92_ensembl_conversion[gene]=[]
    
    result=mygene.MyGeneInfo().query(gene,scopes="symbol",fields=["ensembl"],species="human",verbose=True)
    
    for hit in result['hits']:
        score=hit['_score']
        if score > 1:
            if 'ensembl' in hit:
                if type(hit['ensembl']) == dict:
                    elements=[hit['ensembl']]
                elif type(hit['ensembl']) == list:
                    elements=hit['ensembl']
                else:
                    raise Exception("Error: variable type not recognized.")
                for element in elements:
                    ensembl_ID=element['gene']
                    type_of_gene=element['type_of_gene']
                    emc92_ensembl_conversion[gene].append((ensembl_ID,type_of_gene,score))
            
print(len(emc92_ensembl_conversion.keys()))

90


In [10]:
# map emc92 ensembl IDs to TPM IDs
found_IDs=[]
for element in emc92_ensembl_conversion:
    found=False
    for hit in emc92_ensembl_conversion[element]:
        ensembl_ID=hit[0]  
        if ensembl_ID in tpms.index and found == False:
            found_IDs.append(ensembl_ID)
            found=True
print(len(list(set(found_IDs))))

90


## 1.3. obtain model state classification

In [11]:
# read data
with open(transcriptional_states_file) as json_file:
    states = json.load(json_file)

In [12]:
# remove peripheral blood samples
bm_states={}
for state_ID in states:
    bm=[element for element in states[state_ID] if element.split('_')[-1] == 'BM']
    bm_states[state_ID]=bm

In [13]:
# get a list of BM patients
all_patients=[]
for state_ID in bm_states:
    for patient in bm_states[state_ID]:
        if patient not in all_patients:
            all_patients.append(patient)
print(len(all_patients))

821


In [14]:
# intersect with count headers
intersect=list(set(tpms.columns) & set(all_patients))

# 2. prepare final data structures

## 2.1. expression

In [15]:
print(len(intersect))
df=tpms[intersect]
df=df.loc[found_IDs]
print(df.shape)
df.head()

683
(90, 683)


,MMRF_2407_1_BM,MMRF_2514_1_BM,MMRF_1997_1_BM,MMRF_1814_1_BM,MMRF_1432_1_BM,MMRF_1309_1_BM,MMRF_1694_1_BM,MMRF_1886_1_BM,MMRF_1861_1_BM,MMRF_1574_1_BM,...,MMRF_1630_1_BM,MMRF_2217_1_BM,MMRF_2339_1_BM,MMRF_1229_1_BM,MMRF_1031_1_BM,MMRF_1677_1_BM,MMRF_1494_1_BM,MMRF_1484_1_BM,MMRF_1597_1_BM,MMRF_1815_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000049323,28.32370,2.886430,9.20436,10.31540,0.886305,2.302290,24.631200,1.59414,2.78733,3.868140,...,4.32126,8.58877,30.66950,5.192820,3.421300,4.727100,6.68177,4.469090,12.98170,3.82552
ENSG00000163935,4.49055,2.805050,6.71199,7.44570,5.301470,3.116550,6.580710,6.74790,2.84316,3.333970,...,5.61853,7.94770,8.03161,2.567680,4.634200,4.110130,3.49712,5.765000,10.39750,6.32405
ENSG00000008952,104.83200,88.729800,118.23500,125.82100,77.160400,50.229800,105.191000,75.95570,48.42210,93.876000,...,105.18000,136.56800,196.30500,98.540900,125.764000,91.520300,56.57780,79.799900,291.81600,108.65500
ENSG00000182326,1.37498,0.331174,2.07640,1.91777,1.036560,0.318854,0.786051,1.23112,2.27588,0.357795,...,1.09322,2.55955,0.74942,0.953843,0.080564,0.356468,4.16300,0.468313,2.14389,3.28020
ENSG00000164022,23.28060,21.179000,19.26250,31.46610,32.345400,13.618500,27.543300,17.98000,17.27800,15.137400,...,27.95360,27.82520,29.03850,22.466500,9.832110,19.418800,22.66520,25.327400,39.90490,32.88920


In [16]:
# convert to rank
ranks=df.rank()
print(ranks.shape)
ranks.head()

(90, 683)


,MMRF_2407_1_BM,MMRF_2514_1_BM,MMRF_1997_1_BM,MMRF_1814_1_BM,MMRF_1432_1_BM,MMRF_1309_1_BM,MMRF_1694_1_BM,MMRF_1886_1_BM,MMRF_1861_1_BM,MMRF_1574_1_BM,...,MMRF_1630_1_BM,MMRF_2217_1_BM,MMRF_2339_1_BM,MMRF_1229_1_BM,MMRF_1031_1_BM,MMRF_1677_1_BM,MMRF_1494_1_BM,MMRF_1484_1_BM,MMRF_1597_1_BM,MMRF_1815_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000049323,67.0,31.0,52.0,47.0,12.0,38.0,62.0,17.0,32.0,43.0,...,34.0,43.0,63.0,44.0,23.0,43.0,38.0,35.0,47.0,22.0
ENSG00000163935,44.0,30.0,47.0,37.0,24.0,41.0,36.0,43.0,33.0,40.0,...,41.0,39.0,34.0,27.0,31.0,39.0,22.0,41.0,44.0,32.0
ENSG00000008952,80.0,80.0,80.0,82.0,75.0,79.0,79.0,80.0,75.0,81.0,...,79.0,80.0,82.0,80.0,79.0,79.0,73.0,77.0,83.0,77.0
ENSG00000182326,27.0,17.0,25.0,18.0,14.0,19.0,13.0,13.0,29.0,13.0,...,21.0,20.0,13.0,19.0,8.0,16.0,25.0,17.0,22.0,16.0
ENSG00000164022,63.0,62.0,61.0,68.0,65.0,62.0,64.0,59.0,61.0,62.0,...,62.0,61.0,61.0,67.0,46.0,63.0,60.0,62.0,65.0,63.0


## 2.2. pairs

In [17]:
pairs=[]
for i in range(len(found_IDs)):
    for j in range(len(found_IDs)):
        if i > j:
            pairs.append((found_IDs[i],found_IDs[j]))
print(len(pairs))
print(((len(found_IDs)**2)-len(found_IDs))/2)

4005
4005.0


## 2.3. states

In [18]:
states={}
for state in bm_states:
    states[state]=[]
    for patient in bm_states[state]:
        if patient in intersect:
            states[state].append(patient)

In [19]:
sizes=[]
for state in bm_states:
    print(state,len(bm_states[state]),len(states[state]))
    sizes.append(len(states[state]))
sizes.sort(reverse=True)
print(sizes)

24 2 2
20 41 37
21 13 12
22 13 11
23 29 26
1 96 64
0 104 52
3 92 90
2 30 19
5 20 18
4 31 29
7 36 33
6 6 6
9 6 5
8 17 16
11 54 52
10 10 10
13 18 17
12 23 18
15 43 40
14 22 15
17 30 28
16 62 61
19 4 4
18 19 18
[90, 64, 61, 52, 52, 40, 37, 33, 29, 28, 26, 19, 18, 18, 18, 17, 16, 15, 12, 11, 10, 6, 5, 4, 2]


# 3. find matrices of pair differential occurrence

In [20]:
new={}
for key in states:
    if len(states[key]) > 50:
        new[key]=states[key]

In [33]:
pairs=pairs[:20]

In [44]:
scipy.stats.entropy([1,1,1],base=2)

1.584962500721156

In [36]:
for pair in pairs:
    probabilities=[]
    for state in new:
        r=[]
        for patient in new[state]:
            rankA=ranks.loc[pair[0],patient]
            rankB=ranks.loc[pair[1],patient]
            if rankA > rankB:
                v=1
            else:
                v=0
            r.append(v)
        p=sum(r)/len(r)
        probabilities.append(p)
    print(pair)
    print(probabilities)
    shannon=scipy.stats.entropy(probabilities,base=2)
    print()


('ENSG00000163935', 'ENSG00000049323')
[0.5625, 0.7115384615384616, 0.45555555555555555, 0.5384615384615384, 0.4426229508196721]

('ENSG00000008952', 'ENSG00000049323')
[1.0, 1.0, 1.0, 1.0, 1.0]

('ENSG00000008952', 'ENSG00000163935')
[1.0, 1.0, 1.0, 1.0, 1.0]

('ENSG00000182326', 'ENSG00000049323')
[0.109375, 0.40384615384615385, 0.011111111111111112, 0.038461538461538464, 0.04918032786885246]

('ENSG00000182326', 'ENSG00000163935')
[0.0625, 0.11538461538461539, 0.0, 0.019230769230769232, 0.03278688524590164]

('ENSG00000182326', 'ENSG00000008952')
[0.0, 0.0, 0.0, 0.0, 0.0]

('ENSG00000164022', 'ENSG00000049323')
[0.96875, 1.0, 0.9333333333333333, 0.9423076923076923, 0.9344262295081968]

('ENSG00000164022', 'ENSG00000163935')
[1.0, 1.0, 1.0, 1.0, 1.0]

('ENSG00000164022', 'ENSG00000008952')
[0.0, 0.038461538461538464, 0.0, 0.0, 0.0]

('ENSG00000164022', 'ENSG00000182326')
[1.0, 0.9807692307692307, 1.0, 1.0, 1.0]

('ENSG00000089775', 'ENSG00000049323')
[0.46875, 0.7307692307692307, 0.1

In [22]:
# define the n most entropic pairs, for each state. Then get the call using proportions on information for likelihood. Likelihood might not be needed.

In [23]:
# this is going to be state-specific. At least 5 patients please

In [24]:
# make a confusion matrix of true and predictederk[]


# 3. find biomarkers: state-specific perfect pair sets

# 4. how useful are sets of random genes?